# Query Traffic Counts for Red Light Camera Locations
* Chicago Data Portal dataset (https://data.cityofchicago.org/Transportation/Average-Daily-Traffic-Counts/pfsx-4n4m)
* Calculate daily average ticket per 1,000 vehicles

### Set up dependencies and API parameters

In [1]:
# Import dependencies and key
import requests
import json
import pandas as pd

# API base_url
base_url = "https://data.cityofchicago.org/resource/pfsx-4n4m.json"
max_search_radius = 16000

# camera location file
csv_file = "Chicago_Count_by_Location_LatLng.csv"

### Read geocoded camera location

In [2]:
# Read geocoded data, drop entries without lat/long
df = pd.read_csv(csv_file)
df = df.dropna()
print(f"Number of geocoded locations: {df.shape[0]}")
df.head()

Number of geocoded locations: 465


,address,Total,First Ticket Date,Last Ticket Date,Days,Daily Avg,Lat,Lng,zipcode
0,1 E 63RD ST,12038,3/3/2014,4/27/2018,1517,7.935399,41.780047,-87.625323,60637.0
1,1 E 75TH STREET,15159,3/1/2014,4/27/2018,1519,9.979592,41.758293,-87.616316,60619.0
2,1 E 79TH STREET,38135,3/1/2014,4/27/2018,1519,25.105332,41.750619,-87.624256,60619.0
3,1 N ASHLAND AVE,7229,3/1/2014,4/27/2018,1519,4.759052,41.899593,-87.667609,60622.0
4,1 N HALSTED STREET,5696,3/1/2014,4/27/2018,1519,3.749835,41.882169,-87.646977,60661.0


### Query traffic data using API

In [4]:
print(f"Searching for nearby traffic count data...")

# Loop 1: Search for each address
for index, row in df.iterrows():
    # initial parameters
    radius = 500    # search circle at start
    found = False   # found status
    
    # Loop 2: Search traffic data until found one on the same street 
    #         or found none within maximum search radius
    while (not found) and (radius <= max_search_radius): 
        # url
        url = f"{base_url}?$where=within_circle(location," + \
            f"%20{row['Lat']},%20{row['Lng']},%20{radius})"
    
        # make request
        results = requests.get(url).json()
        
        # loop 3: Go through returned results to find one on the same street 
        count = len(results)
        for item in results:
            if (row['address'].split()[2].lower()) in (item['street'].lower()):
                try:
                    print(f"Found a match for {row['address']}")
                    found = True
                    df.loc[index, 'Traffic Loc Found'] = count
                    df.loc[index, 'Radius'] = radius
                    df.loc[index, 'Traffic Address'] = \
                        f"{item['traffic_volume_count_location_address']} " + \
                        f"{item['street']}"
                    df.loc[index, 'Date_count'] = item['date_of_count']
                    df.loc[index, 'Total Volume'] = item['total_passing_vehicle_volume']
                    df.loc[index, 'Vol by Direction'] = \
                        item['vehicle_volume_by_each_direction_of_traffic']
                    break
                except:
                    print("Error importing traffic data")
        
        # increase searching radius if not found, run Loop 2 again
        if not found:
            radius *= 2
            
    # display message if traffic data not found
    if not found:
        print(f"Cannot locate traffic data for {row['address']}")

print('----------------------')
print('Done')
df.head()

Searching for nearby traffic count data...
Found a match for 1 E 63RD ST
Found a match for 1 E 75TH STREET
Found a match for 1 E 79TH STREET
Found a match for 1 N ASHLAND AVE
Found a match for 1 N HALSTED STREET
Found a match for 1 N HAMLIN AVENUE
Found a match for 1 S CENTRAL AVENUE
Found a match for 1 S HALSTED STREET
Found a match for 1 S WESTERN AVE
Found a match for 100 E JACKSON BLVD
Found a match for 100 E ONTARIO ST
Found a match for 100 N CICERO AVENUE
Found a match for 100 W CHICAGO AVENUE
Found a match for 1000 W FOSTER AVE
Found a match for 1000 W HOLLYWOOD AVE
Found a match for 10300 S HALSTED STREE
Found a match for 10318 S INDIANAPOLIS
Found a match for 1110 S PULASKI RD
Found a match for 11100 S HALSTED ST
Found a match for 11100 S VINCENNES AVE
Cannot locate traffic data for 1111 N HUMBOLDT
Found a match for 11144 S VINCENNES
Found a match for 11153 S VINCENNES
Found a match for 1117 S PULASKI RD
Found a match for 1142 W IRVING PARK
Found a match for 115 N OGDEN
Found 

,address,Total,First Ticket Date,Last Ticket Date,Days,Daily Avg,Lat,Lng,zipcode,Traffic Loc Found,Radius,Traffic Address,Date_count,Total Volume,Vol by Direction
0,1 E 63RD ST,12038,3/3/2014,4/27/2018,1517,7.935399,41.780047,-87.625323,60637.0,3.0,500.0,120 East 63rd St,2006-03-28T08:00:00.000Z,12400,East Bound: 7800 / West Bound: 4600
1,1 E 75TH STREET,15159,3/1/2014,4/27/2018,1519,9.979592,41.758293,-87.616316,60619.0,5.0,500.0,504 East 75th St,2006-03-28T08:00:00.000Z,16100,East Bound: 8500 / West Bound: 7600
2,1 E 79TH STREET,38135,3/1/2014,4/27/2018,1519,25.105332,41.750619,-87.624256,60619.0,2.0,500.0,20 East 79th St,2006-03-28T08:00:00.000Z,20700,East Bound: 11300 / West Bound: 9400
3,1 N ASHLAND AVE,7229,3/1/2014,4/27/2018,1519,4.759052,41.899593,-87.667609,60622.0,7.0,500.0,755 North Ashland Ave,2006-10-26T07:00:00.000Z,30500,North Bound: 15800 / South Bound: 14700
4,1 N HALSTED STREET,5696,3/1/2014,4/27/2018,1519,3.749835,41.882169,-87.646977,60661.0,14.0,500.0,159 North Halsted St,2006-10-02T07:00:00.000Z,20000,North Bound: 10200 / South Bound: 9800


In [11]:
# Normalize ticket count by traffic count
df['Total Volume'] = df['Total Volume'].astype('float')
df['Ticket per 1000 AADT'] = df['Daily Avg'] / (df['Total Volume'] / 2) * 1000
df.head()

,address,Total,First Ticket Date,Last Ticket Date,Days,Daily Avg,Lat,Lng,zipcode,Traffic Loc Found,Radius,Traffic Address,Date_count,Total Volume,Vol by Direction,Ticket per 1000 AADT
0,1 E 63RD ST,12038,3/3/2014,4/27/2018,1517,7.935399,41.780047,-87.625323,60637.0,3.0,500.0,120 East 63rd St,2006-03-28T08:00:00.000Z,12400.0,East Bound: 7800 / West Bound: 4600,1.279903
1,1 E 75TH STREET,15159,3/1/2014,4/27/2018,1519,9.979592,41.758293,-87.616316,60619.0,5.0,500.0,504 East 75th St,2006-03-28T08:00:00.000Z,16100.0,East Bound: 8500 / West Bound: 7600,1.239701
2,1 E 79TH STREET,38135,3/1/2014,4/27/2018,1519,25.105332,41.750619,-87.624256,60619.0,2.0,500.0,20 East 79th St,2006-03-28T08:00:00.000Z,20700.0,East Bound: 11300 / West Bound: 9400,2.425636
3,1 N ASHLAND AVE,7229,3/1/2014,4/27/2018,1519,4.759052,41.899593,-87.667609,60622.0,7.0,500.0,755 North Ashland Ave,2006-10-26T07:00:00.000Z,30500.0,North Bound: 15800 / South Bound: 14700,0.312069
4,1 N HALSTED STREET,5696,3/1/2014,4/27/2018,1519,3.749835,41.882169,-87.646977,60661.0,14.0,500.0,159 North Halsted St,2006-10-02T07:00:00.000Z,20000.0,North Bound: 10200 / South Bound: 9800,0.374984


In [12]:
# save results to csv
df.to_csv("Chicago_Count_AADT_by_Location.csv")